In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn import preprocessing
from sklearn import svm, model_selection

In [ ]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/main_merged1.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
merged_df.drop(['Pos', 'Date', 'W_PTS', 'L_PTS', 'DD', 'TD',
       'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'Starter',
       'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'MP', 'FG', 'FGA',
               'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'BLK Last Week', 'BLK Last Month', 'TRB_AVG', 'STL_AVG',
       'FT_perc', 'TRB', 'STL', 'BLK', 'TOV', 'PF', 'W', 'BLK_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG', 'FG_AVG Last Week',
       'FGA_AVG Last Week', 'FG_perc_AVG Last Week', 'FG_AVG Last Month', 'AST Last Week', 'AST Last Month', 'AST', 'AST_AVG',
       'FGA_AVG Last Month', 'FG_perc_AVG Last Month'], axis=1, inplace=True)

In [ ]:
print(merged_df.columns)
print(merged_df.dtypes)
print(merged_df.shape)
print(merged_df)

In [ ]:
merged_df['Team'] = pd.Categorical(merged_df['Team'])
merged_df['Opp'] = pd.Categorical(merged_df['Opp'])
dummies_team = pd.get_dummies(merged_df['Team'], prefix = 'Team')
dummies_opp = pd.get_dummies(merged_df['Opp'], prefix = 'Opp')
merged_df = pd.concat([merged_df, dummies_team], axis=1)
merged_df = pd.concat([merged_df, dummies_opp], axis=1)

In [ ]:
merged_df['Name'] = pd.Categorical(merged_df['Name'])
dummies_name = pd.get_dummies(merged_df['Name'], prefix = '')
merged_df = pd.concat([merged_df, dummies_name], axis=1)
merged_df

In [ ]:
train = merged_df.loc[1609:25000]
test = merged_df.loc[25000:25500]
X_train = train.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
y_train = train[['PTS']]
X_test = test.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
y_test = test[['PTS']]

In [ ]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)
parameters = [{'kernel': ['rbf'], 'gamma': [0.01, 0.1, 0.2, 1e-3, 0,5], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                   {'kernel': ['poly'], 'gamma': [0.01, 0.1, 0.2, 1e-3, 0,5], 'C': [1, 10, 100, 1000], 'degree' : [1.5, 2, 3]},
                   {'kernel': ['sigmoid'], 'gamma': [0.01, 0.1, 0.2, 1e-3, 0,5], 'C': [1, 10, 100, 1000]}]

cv_svr = model_selection.GridSearchCV(svm.SVR(), parameters, cv = 3, n_jobs = -1, scoring = scorer)
cv_svr.fit(X_train, y_train)
cv_svr.best_param_

In [ ]:
svr1 = svm.SVR(kernel='poly', degree=2, epsilon=0.1, gamma=0.2, C=10)
svr1.fit(X_train, y_train)

In [ ]:
# 34.34100673679344 0.5640552429661726

y_pred = svr.predict(X_test)
print(mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))